In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [17]:
df=spark.read.csv("original.csv",header=True)
df2=spark.read.format('csv').option('header','true').load('original.csv')
#ways of loading the data

In [35]:
df=spark.read.csv("original.csv",header=True)

In [5]:
df.show()
df2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [9]:
df.dtypes

[('id', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string')]

In [44]:
from pyspark.sql.types import *
schema=StructType([
    StructField('id',IntegerType()),
    StructField('first_name',StringType()),
    StructField('last_name',StringType()),
    StructField('gender',StringType()),
    StructField('city',StringType()),
    StructField('job_title',StringType()),
    StructField('Salary',StringType()),
    StructField('latitude',FloatType()),
    StructField('longitude',FloatType())
])
df = spark.read.csv('original.csv',header=True,schema=schema)

In [45]:
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           city|           job_title|   Salary|  latitude| longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [47]:
df_select=df.select('first_name','last_name')

In [48]:
df_select.show()

+----------+----------+
|first_name| last_name|
+----------+----------+
|   Melinde| Shilburne|
|  Kimberly|Von Welden|
|    Alvera|  Di Boldi|
|   Shannon| O'Griffin|
|  Sherwood|   Macieja|
|     Maris|      Folk|
|     Masha|    Divers|
|   Goddart|     Flear|
|      Roth|O'Cannavan|
|      Bran|   Trahear|
|    Kylynn|   Lockart|
|       Rey|    Meharg|
|      Kerr|    Braden|
|    Mickie| Whanstall|
|    Kaspar|     Pally|
|    Norbie|    Gwyllt|
|    Claude|    Briant|
|     Thain|    Habbon|
|  Tiffanie|  Pattison|
|    Ettore|  Gerriets|
+----------+----------+
only showing top 20 rows



In [49]:
df_renamed=df.withColumnRenamed('first_name','fn')

In [50]:
df_renamed.show()

+---+--------+----------+------+---------------+--------------------+---------+----------+----------+
| id|      fn| last_name|gender|           city|           job_title|   Salary|  latitude| longitude|
+---+--------+----------+------+---------------+--------------------+---------+----------+----------+
|  1| Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|  Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4| Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|   Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|   Masha|    Divers|Female|         Dachun|                null|$25090.87| 24

In [51]:
df_filter=df.filter((df.first_name=='Alvera'))

In [52]:
df_filter.show()

+---+----------+---------+------+----+---------+---------+---------+----------+
| id|first_name|last_name|gender|city|job_title|   Salary| latitude| longitude|
+---+----------+---------+------+----+---------+---------+---------+----------+
|  3|    Alvera| Di Boldi|Female|null|     null|$57576.52|39.994747|116.339775|
+---+----------+---------+------+----+---------+---------+---------+----------+



In [57]:
df_filter=df.filter((df.first_name.like('%lver%')))

In [58]:
df_filter.show()

+---+----------+---------+------+----------+-------------------+---------+---------+----------+
| id|first_name|last_name|gender|      city|          job_title|   Salary| latitude| longitude|
+---+----------+---------+------+----------+-------------------+---------+---------+----------+
|  3|    Alvera| Di Boldi|Female|      null|               null|$57576.52|39.994747|116.339775|
|775|   Alverta| MacNulty|Female|Megalópoli|Geological Engineer|$17299.62|37.401245| 22.136488|
+---+----------+---------+------+----------+-------------------+---------+---------+----------+



In [59]:
df_filter=df.filter((df.first_name.endswith('din')))

In [60]:
df_filter.show()

+---+----------+-------------+------+-----------+---------+---------+----------+---------+
| id|first_name|    last_name|gender|       city|job_title|   Salary|  latitude|longitude|
+---+----------+-------------+------+-----------+---------+---------+----------+---------+
|901|     Aldin|Matuszkiewicz|  Male|East London| Operator|$41468.83|-32.954933|27.931913|
+---+----------+-------------+------+-----------+---------+---------+----------+---------+



In [62]:
df_filter=df.filter((df.first_name.startswith('Alv')))

In [63]:
df_filter.show()

+---+----------+---------+------+----------+--------------------+---------+---------+----------+
| id|first_name|last_name|gender|      city|           job_title|   Salary| latitude| longitude|
+---+----------+---------+------+----------+--------------------+---------+---------+----------+
|  3|    Alvera| Di Boldi|Female|      null|                null|$57576.52|39.994747|116.339775|
| 81|     Alvin|    Doman|  Male|      Niny|Research Assistant I|$53258.86|44.486843| 43.940807|
|775|   Alverta| MacNulty|Female|Megalópoli| Geological Engineer|$17299.62|37.401245| 22.136488|
+---+----------+---------+------+----------+--------------------+---------+---------+----------+



In [64]:
df_filter = df.filter((df.id.between(1,5)))

In [65]:
df_filter.show()

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
| id|first_name| last_name|gender|         city|           job_title|   Salary| latitude| longitude|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|     null| 37.648994|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+



In [66]:
df_filter = df.filter((df.first_name.isin('Aldin','Valma')))

In [67]:
df_filter.show()

+---+----------+-------------+------+-----------+----------------+---------+----------+---------+
| id|first_name|    last_name|gender|       city|       job_title|   Salary|  latitude|longitude|
+---+----------+-------------+------+-----------+----------------+---------+----------+---------+
|569|     Valma|      Bratton|Female|  Kurayoshi|Web Developer II|$32665.89| 35.449905|133.76134|
|901|     Aldin|Matuszkiewicz|  Male|East London|        Operator|$41468.83|-32.954933|27.931913|
+---+----------+-------------+------+-----------+----------------+---------+----------+---------+



In [69]:
df_substr=df.select(df.first_name, df.first_name.substr(1,5).alias('name'))

In [70]:
df_substr.show()

+----------+-----+
|first_name| name|
+----------+-----+
|   Melinde|Melin|
|  Kimberly|Kimbe|
|    Alvera|Alver|
|   Shannon|Shann|
|  Sherwood|Sherw|
|     Maris|Maris|
|     Masha|Masha|
|   Goddart|Godda|
|      Roth| Roth|
|      Bran| Bran|
|    Kylynn|Kylyn|
|       Rey|  Rey|
|      Kerr| Kerr|
|    Mickie|Micki|
|    Kaspar|Kaspa|
|    Norbie|Norbi|
|    Claude|Claud|
|     Thain|Thain|
|  Tiffanie|Tiffa|
|    Ettore|Ettor|
+----------+-----+
only showing top 20 rows



In [75]:
df_filter=df.filter((df.first_name.isin('Aldin','Valma')) | (df.city.like('%ondon')))

In [76]:
df_filter.show()

+---+----------+-------------+------+-----------+----------------+---------+----------+---------+
| id|first_name|    last_name|gender|       city|       job_title|   Salary|  latitude|longitude|
+---+----------+-------------+------+-----------+----------------+---------+----------+---------+
|569|     Valma|      Bratton|Female|  Kurayoshi|Web Developer II|$32665.89| 35.449905|133.76134|
|901|     Aldin|Matuszkiewicz|  Male|East London|        Operator|$41468.83|-32.954933|27.931913|
+---+----------+-------------+------+-----------+----------------+---------+----------+---------+



In [77]:
df_filter = df.filter((df.id>10) & (df.id<100))

In [79]:
df_filter.show()

+---+----------+---------+------+--------------+--------------------+---------+----------+----------+
| id|first_name|last_name|gender|          city|           job_title|   Salary|  latitude| longitude|
+---+----------+---------+------+--------------+--------------------+---------+----------+----------+
| 11|    Kylynn|  Lockart|Female|      El Cardo|Nuclear Power Eng...|$13604.63|     -5.85| -79.88333|
| 12|       Rey|   Meharg|Female|   Wangqingtuo|Systems Administr...|$73423.70|  39.17238| 116.93161|
| 13|      Kerr|   Braden|  Male|     Sułkowice|Compensation Analyst|$33432.99|  49.81518| 19.377174|
| 14|    Mickie|Whanstall|  Male|   Springfield|Assistant Media P...|$50838.53|  42.10148|-72.576675|
| 15|    Kaspar|    Pally|  Male|        Chrást|  Analyst Programmer|$40163.03|  49.79233| 13.491532|
| 16|    Norbie|   Gwyllt|  Male|        Xijiao|              Editor|$32492.73| 43.494576|  5.897802|
| 17|    Claude|   Briant|Female|     Mieścisko|Research Assistan...|$51862.48| 52